#  Diseño de Filtros IIR por Transformación Bilineal

## Conceptos Clave

Según la Clase 16, el diseño de filtros IIR (Infinite Impulse Response) a menudo se basa en el diseño de filtros analógicos bien establecidos y luego se transforman en filtros digitales utilizando una **transformación del plano S al plano Z**. La **transformación bilineal** es un método estándar para esta conversión:
$$s = \frac{1-z^{-1}}{1+z^{-1}}$$
Esta transformación mapea el eje $j\Omega$ del plano S (frecuencias analógicas) al círculo unitario $|z|=1$ del plano Z (frecuencias digitales).

Una consecuencia importante de la transformación bilineal es el **alineamiento de frecuencias** o "frequency warping". La relación entre la frecuencia analógica $\Omega$ y la frecuencia digital $\omega$ no es lineal:
$$\Omega = \tan\left(\frac{\omega}{2}\right)$$
Esto significa que las frecuencias en el dominio analógico se "comprimen" a medida que nos acercamos a frecuencias analógicas infinitas, mapeándose al rango de frecuencias digitales de $0$ a $\pi$. Para diseñar un filtro digital con una frecuencia de corte deseada $\omega_c$, es necesario **pre-distorsionar** (pre-warp) la frecuencia de corte analógica a $\Omega_c$ utilizando la relación inversa:
$$\Omega_c = \tan\left(\frac{\omega_c}{2}\right)$$
El diseño del filtro analógico se realiza entonces con esta frecuencia de corte pre-distorsionada $\Omega_c$. Una vez obtenido el filtro analógico $H_a(s)$, se aplica la transformación bilineal $s = \frac{1-z^{-1}}{1+z^{-1}}$ para obtener el filtro digital $H(z) = H_a(s)|_{s=\frac{1-z^{-1}}{1+z^{-1}}}$.

## Ejemplo 1: Diseño de un Filtro Pasabajos de Primer Orden por Transformación Bilineal

Este ejemplo sigue el procedimiento de diseño de un filtro pasabajos de primer orden utilizando la transformación bilineal, tal como se presenta en la Clase 16. Se parte de un filtro analógico de primer orden y se calculan los coeficientes del filtro digital equivalente.

Consideraremos el diseño de un filtro pasabajos digital con las siguientes especificaciones:
* Frecuencia de muestreo digital ($f_s$): 10 kHz
* Frecuencia de corte de 3 dB digital ($f_c$): 1 kHz

El filtro pasabajos analógico de primer orden base tiene una función de transferencia de la forma $H_a(s) = \frac{\alpha}{s + \alpha}$. Para una frecuencia de corte de 3 dB, la frecuencia de corte analógica $\Omega_c$ es igual a $\alpha$.

Los pasos son:
1.  Convertir la frecuencia de corte digital $f_c$ a radianes/muestra $\omega_c = \frac{2\pi f_c}{f_s}$.
2.  Calcular la frecuencia de corte analógica pre-distorsionada $\Omega_c = \tan(\omega_c/2)$.
3.  Como la frecuencia de corte analógica de 3 dB es $\Omega_c$, el parámetro del filtro analógico es $\alpha = \Omega_c$.
4.  Aplicar la transformación bilineal a $H_a(s) = \frac{\alpha}{s + \alpha}$ para obtener $H(z)$. La Clase 16 muestra que esto resulta en un filtro digital de la forma $H(z) = b \frac{1+z^{-1}}{1 - az^{-1}}$, con coeficientes $a = \frac{1-\alpha}{1+\alpha}$ y $b = \frac{\alpha}{1+\alpha}$.

In [1]:
# --------------- Imports ---------------
import numpy as np
import math

# --- Ejemplo 1: Diseño de Filtro Pasabajos de Primer Orden IIR ---
print("--- Ejemplo 1: Diseño de Filtro Pasabajos IIR por Transformación Bilineal ---")

# Especificaciones del filtro digital
fs = 10000 # Frecuencia de muestreo (Hz)
fc = 1000  # Frecuencia de corte digital de 3 dB (Hz)

print(f"Frecuencia de muestreo (fs): {fs} Hz")
print(f"Frecuencia de corte digital deseada (fc): {fc} Hz")

# 1. Convertir fc a frecuencia digital en radianes/muestra (ωc)
omega_c = (2 * np.pi * fc) / fs
print(f"Frecuencia de corte digital (ωc): {omega_c:.4f} rad/muestra ({omega_c/np.pi:.4f}π)")

# 2. Calcular la frecuencia de corte analógica pre-distorsionada (Ωc)
Omega_c = np.tan(omega_c / 2)
print(f"Frecuencia de corte analógica pre-distorsionada (Ωc): {Omega_c:.4f} rad/segundo (análogo)")

# 3. Para un filtro pasabajos analógico de primer orden de 3 dB, α = Ωc
alpha = Omega_c
print(f"Parámetro del filtro analógico (α): {alpha:.4f}")

# 4. Calcular los coeficientes del filtro digital (a y b)
# H(z) = b * (1 + z^-1) / (1 - a * z^-1)
a = (1 - alpha) / (1 + alpha)
b = alpha / (1 + alpha)

print(f"\nCoeficientes del filtro digital IIR (H(z) = b * (1 + z^-1) / (1 - a * z^-1)):")
print(f"  a = {a:.6f}")
print(f"  b = {b:.6f}")

print("\n--- Fin del Ejemplo 1 ---\n")

--- Ejemplo 1: Diseño de Filtro Pasabajos IIR por Transformación Bilineal ---
Frecuencia de muestreo (fs): 10000 Hz
Frecuencia de corte digital deseada (fc): 1000 Hz
Frecuencia de corte digital (ωc): 0.6283 rad/muestra (0.2000π)
Frecuencia de corte analógica pre-distorsionada (Ωc): 0.3249 rad/segundo (análogo)
Parámetro del filtro analógico (α): 0.3249

Coeficientes del filtro digital IIR (H(z) = b * (1 + z^-1) / (1 - a * z^-1)):
  a = 0.509525
  b = 0.245237

--- Fin del Ejemplo 1 ---



## Ejemplo Extra: Visualización Interactiva del Alineamiento de Frecuencias (Frequency Warping)

Este ejemplo interactivo te permite visualizar la relación no lineal entre la frecuencia analógica $\Omega$ (en el plano S) y la frecuencia digital $\omega$ (en el plano Z) introducida por la transformación bilineal:

$$\Omega = \tan\left(\frac{\omega}{2}\right)$$

Esta relación muestra cómo las frecuencias del dominio analógico de $0$ a infinito se mapean al rango de frecuencias digitales de $0$ a $\pi$. Debido a esta compresión no lineal, si deseas que tu filtro digital tenga una frecuencia de corte específica $\omega_c$, debes diseñar el filtro analógico correspondiente con una frecuencia de corte **pre-distorsionada** $\Omega_c$ tal que $\Omega_c = \tan(\omega_c/2)$.

Ajusta el slider para seleccionar una frecuencia de corte digital deseada ($\omega_c$, normalizada por $\pi$) y observa en la gráfica:
* La curva que representa la relación $\Omega = \tan(\omega/2)$.
* El punto $(\omega_c/\pi, \Omega_c)$ resaltado, mostrando la frecuencia analógica pre-distorsionada necesaria para lograr la frecuencia de corte digital deseada.

Esto ilustra visualmente el fenómeno de "frequency warping" y la razón detrás de la necesidad de pre-distorsionar las frecuencias al usar la transformación bilineal en el diseño de filtros IIR.

### Código Python para Ejemplo Extra (Interactivo)

In [3]:
# --------------- Imports ---------------
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import FloatSlider, VBox, interactive_output, Layout
from IPython.display import display, clear_output
import math

# --- Función para calcular y graficar el alineamiento de frecuencias ---

def plot_frequency_warping(omega_c_norm):
    """
    Grafica la relación Omega vs omega y resalta el punto de frecuencia de corte.
    omega_c_norm es la frecuencia de corte digital normalizada entre 0 y 1 (correspondiente a 0 a pi).
    """
    # Limpiar la salida anterior
    with output_area:
        clear_output(wait=True)

    print(f"--- Alineamiento de Frecuencias de la Transformación Bilineal ---")

    # Vector de frecuencias digitales (ω) de 0 a π
    # Usamos un pequeño valor epsilon al final para evitar np.tan(pi/2)
    omega = np.linspace(0, np.pi, 200, endpoint=False) # Evitar pi exactamente

    # Calcular la frecuencia analógica correspondiente (Ω)
    Omega = np.tan(omega / 2)

    # Calcular la frecuencia de corte digital y su correspondiente frecuencia analógica
    omega_c = omega_c_norm * np.pi
    Omega_c = np.tan(omega_c / 2)

    print(f"Frecuencia de corte digital seleccionada (ωc): {omega_c:.4f} rad/muestra ({omega_c_norm:.2f}π)")
    print(f"Frecuencia analógica pre-distorsionada correspondiente (Ωc): {Omega_c:.4f}")

    # --- Creación de la figura ---
    plt.figure(figsize=(10, 6))

    # Graficar la relación Ω vs ω
    # Graficar en frecuencia digital normalizada (ω / π)
    plt.plot(omega / np.pi, Omega, label='$\\Omega = \tan(\omega/2)$')

    # Resaltar el punto de frecuencia de corte seleccionado
    plt.plot(omega_c / np.pi, Omega_c, 'ro', markersize=8, label=f'($\\omega_c = {omega_c_norm:.2f}\pi$, $\\Omega_c = {Omega_c:.2f}$)')

    # Añadir líneas de referencia
    plt.axvline(omega_c / np.pi, color='red', linestyle='--', linewidth=0.8)
    plt.axhline(Omega_c, color='red', linestyle='--', linewidth=0.8)


    plt.title('Alineamiento de Frecuencias: $\\Omega$ vs $\\omega$ (Transformación Bilineal)')
    plt.xlabel("Frecuencia digital normalizada ($\omega / \pi$)")
    plt.ylabel("Frecuencia analógica ($\Omega$)")
    plt.grid(True)
    plt.legend()
    plt.xlim(0, 1) # Frecuencia digital de 0 a π (normalizada a 0 a 1)
    # Ajustar límite Y a un valor razonable para visualizar la curva, Ω va a infinito
    plt.ylim(0, 15) # Mostrar hasta Ω = 15 como un límite práctico para visualización


    # Mostrar el gráfico dentro del contexto del output_area
    with output_area:
        plt.show()

    print("\n--- Fin del Gráfico ---")


# --- Widget de Control ---
style = {'description_width': 'initial'}
layout_slider = Layout(width='80%')

omega_c_slider_norm = FloatSlider(
    min=0.01, max=0.99, step=0.01, value=0.2, # Frecuencia de corte digital normalizada (0 a 1, evitando los extremos)
    description='Frec. de Corte Digital ($\omega_c / \pi$):',
    style=style,
    layout=layout_slider,
    readout_format=".2f"
)


# --- Contenedor de Salida ---
output_area = widgets.Output()

# --- Conectar Widget y Mostrar ---
interactive_plot = interactive_output(
    plot_frequency_warping,
    {'omega_c_norm': omega_c_slider_norm}
)

print("Ajusta el slider para ver el alineamiento de frecuencias para diferentes frecuencias de corte digitales:")

display(VBox([omega_c_slider_norm, output_area]))

# Ejecutar la función inicialmente para mostrar el estado por defecto
with output_area:
     plot_frequency_warping(omega_c_slider_norm.value)

print("\n--- Fin del Código Interactivo ---\n")

<>:41: SyntaxWarning: invalid escape sequence '\o'
<>:44: SyntaxWarning: invalid escape sequence '\p'
<>:52: SyntaxWarning: invalid escape sequence '\o'
<>:53: SyntaxWarning: invalid escape sequence '\O'
<>:74: SyntaxWarning: invalid escape sequence '\o'
<>:41: SyntaxWarning: invalid escape sequence '\o'
<>:44: SyntaxWarning: invalid escape sequence '\p'
<>:52: SyntaxWarning: invalid escape sequence '\o'
<>:53: SyntaxWarning: invalid escape sequence '\O'
<>:74: SyntaxWarning: invalid escape sequence '\o'
C:\Users\maxim\AppData\Local\Temp\ipykernel_25260\2822887155.py:41: SyntaxWarning: invalid escape sequence '\o'
  plt.plot(omega / np.pi, Omega, label='$\\Omega = \tan(\omega/2)$')
C:\Users\maxim\AppData\Local\Temp\ipykernel_25260\2822887155.py:44: SyntaxWarning: invalid escape sequence '\p'
  plt.plot(omega_c / np.pi, Omega_c, 'ro', markersize=8, label=f'($\\omega_c = {omega_c_norm:.2f}\pi$, $\\Omega_c = {Omega_c:.2f}$)')
C:\Users\maxim\AppData\Local\Temp\ipykernel_25260\2822887155.py

Ajusta el slider para ver el alineamiento de frecuencias para diferentes frecuencias de corte digitales:



--- Fin del Código Interactivo ---

